# Russian language dataset

In [1]:
from fastai.basics import *
from multiprocessing import Pool
import regex as re
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

ModuleNotFoundError: No module named 'regex'

In [ ]:
NEW_LINE = '<|n|>'

In [ ]:
librusec = '/home/u/nas/librusec/lib.rus.ec'
tmp = './tmp/fb2'
data = Path('../data/full')

In [ ]:
!mkdir ../data
!mkdir ../data/full
!mkdir ../data/classic
!mkdir tmp
!mkdir tmp/fb2
!mkdir tmp/txt

### unpack zips

In [ ]:
zips = get_files(librusec, '.zip')

In [ ]:
def unpack(fn):
    !unzip {fn} -d {tmp} >>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(unpack, zips), len(zips)):
    pass

### convert fb2 to txt

In [ ]:
fbs = get_files(tmp, '.fb2')

In [ ]:
def convert_fb2(fn):
    !xsltproc FB2_2_txt.xsl {fn} > {str(fn).replace('.fb2','.txt').replace('/fb2','/txt')} 2>>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(convert_fb2, fbs), len(fbs)):
    pass

### more data

In [ ]:
!rm -R /share/CE_CACHEDEV1_DATA/data/classic_lit/*
!cp -R /share/CE_CACHEDEV1_DATA/data/downloads/txt /share/CE_CACHEDEV1_DATA/data/classic_lit/

In [ ]:
fbs = get_files('/home/u/nas/classic_lit/txt/', '.fb2', recurse=True); len(fbs)

In [ ]:
!rm -R /home/u/nas/classic_lit/out

In [ ]:
path_out = Path('/home/u/nas/classic_lit/out'); path_out.mkdir(exist_ok = True)

In [ ]:
for fn in fbs:
    nn = (str(fn.name)
        .replace(' ','.')
        .replace('_quot;','.')
        .replace('!','.')
        .replace(',','.')
        .replace('(','.')
        .replace(')','.')
        .replace('\xa0','.')
         )
    shutil.move(fn, path_out/nn)

In [ ]:
fbs = get_files(path_out, '.fb2'); len(fbs)

In [ ]:
!rm ../data/classic/*

In [ ]:
data = Path('../data/classic')

In [ ]:
def convert_fb2(fn):
    name = str(fn.name).replace('.fb2','.txt')
    !xsltproc FB2_2_txt.xsl {fn} > {data/name} 2>>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(convert_fb2, fbs), len(fbs)):
    pass

In [ ]:
!rm ../data/classic/*месяцеслов*

In [ ]:
!mkdir ../data/classic/valid

In [ ]:
mv ../data/classic/Tolstoy_Dva_pisma_k_M_Gandi.56185.txt ../data/classic/valid

In [ ]:
mv ../data/classic/Tolstoy_Sobranie_sochineniy_v_dvadtsati_dvuh_tomah_22_Tom_22._Izbrannyie_dnevniki_1895-1910.142868.txt ../data/classic/valid

In [ ]:
mv ../data/classic/Tolstoy_Sobranie_sochineniy_v_dvadtsati_dvuh_tomah_20_Tom_20._Izbrannyie_pisma_1900-1910.142866.txt ../data/classic/valid

### filter txts

In [ ]:
txts = get_files('./tmp/txt', '.txt')

In [ ]:
len(txts)

In [ ]:
# test
fn = txts[0]
match = re.compile('(.|\s)\\1\\1+')
with open(fn, 'r') as f:
    lines = f.read()
lines = 'asdf aaaa dddddfffff' + lines
lines = match.sub(r'\1'*3, lines)             
lines[:1000]

In [ ]:
# test
detect(lines)

In [ ]:
# test
expr = re.compile('([^\n]{150,})([.] )([^\n]{150,})')
while expr.search(lines):
    lines = expr.sub(r'\g<1>.\n\g<3>', lines, 1)
lines[:1000]

In [ ]:
# this will take time, bcs langdetect fails on multithreading
for fn in progress_bar(txts):
    with open(fn, 'r') as f:
        lines = f.read()
        try:
            if len(lines) > 1e+4 and detect(lines) == 'ru':
                with open(f'{data}/{fn.name}', 'w') as c:
                    c.write(lines)
        except LangDetectException as e:
            pass

In [ ]:
txts = get_files(data, '.txt')

In [ ]:
txts += ['../data/poetry_base.txt']

In [ ]:
txts += get_files('../data/classic', '.txt', recurse=True)

In [ ]:
# Add space before each word. It's not really nesessary. 
# It just makes encoding a bit more meaningful to the model and the text smaller(after encoding).

def process_fn(fn):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(fn, 'r') as f:
        lines = f.read()
    if lines and lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    with open(fn, 'w') as c:
        c.write(lines)
        
for _ in progress_bar(Pool(64).imap_unordered(process_fn, txts), len(txts)):
    pass

In [ ]:
with open('../data/poetry_base.txt', 'r') as f:
    lines = f.readlines()
    
split = int(len(lines)*0.95)

with open('../data/poetry_dry.txt', 'w') as f:
    f.writelines(lines[:split])

with open('../data/poetry_eval.txt', 'w') as f:
    f.writelines(lines[split:])

### concat for vocab

Take smallest files to increase word diversity

In [ ]:
txts = get_files(data, '.txt'); len(txts)

In [ ]:
for fn in txts:
    if os.path.getsize(fn) <= 1e+3: 
        os.remove(fn)

In [ ]:
txts = get_files(data, '.txt'); len(txts)

In [ ]:
fsorted = get_files('../data/classic', '.txt') + sorted(txts, key=lambda fn: os.path.getsize(fn))  

In [ ]:
sz=0
with open('./tmp/russian_corpus_for_vocab.txt', 'w') as c:
    for fn in fsorted:
        with open(fn, 'r') as f:
            sz += c.write(f.read().replace('\n', f' {NEW_LINE} ') + '\n')
            if sz > 5e+9:
                break

### cache tokenization (optional)

In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
from run_lm_finetuning import TextDataset
from yt_encoder import YTEncoder

In [ ]:
txts = get_files(data, '.txt'); len(txts)

In [ ]:
def cache_fn(fn):
    tokenizer = YTEncoder.from_pretrained('../bpe/yt.model')
    TextDataset.process_file(fn, tokenizer, 1024, shuffle=True)

In [ ]:
for _ in progress_bar(Pool(32).imap_unordered(cache_fn, txts), len(txts)):
    pass

# Prepare cached dataset for upload (for GCloud)

In [ ]:
files = get_files('upload', '.txt', True); len(files)

In [ ]:
files = [item for item in files if '/full' in str(item) and '/cached' not in str(item)]; len(files)

In [ ]:
for item in files:
    with open(item, 'w'):
        pass